# Binary Classification: to do

- Import the data
- Split the dataset
- Train classifiers
- k-fold validation
- Outcomes
- Documentation and report writing

In [1]:
# Importing data
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

warnings.filterwarnings(action='ignore')

with open('FINAL-OUTPUT.txt') as f:
    full_data = [line.rstrip('\n') for line in f]
"""with open('MEN-OUTPUT.txt') as f:
    man_data = [line.rstrip('\n') for line in f]
with open('WOMEN-OUTPUT.txt') as f:
    woman_data = [line.rstrip('\n') for line in f]"""
print("Total datapoints:", len(full_data))
#print("Ask Men datapoints:", len(man_data))
#print("Ask Women datapoints:", len(woman_data))
#print("sum(AM + AW):", len(woman_data) + len(man_data))

Total datapoints: 1572904
Ask Men datapoints: 809290
Ask Women datapoints: 763614
sum(AM + AW): 1572904


In [2]:
# Organise data
import json

# Total dataset
gender = []
body = []
AMbody = []
AMgender = []
AWbody = []
AWgender = []
subreddit = []
for line in full_data:
    temp = json.loads(line)
    body.append(temp['body'])
    
    
    if (temp['author_flair_css_class'] == 'male'):
        gender.append(1)
    else:
        gender.append(0)
        
    if (temp['subreddit'] == 'AskMen'):
        subreddit.append('AM')
        AMbody.append(temp['body'])
        if (temp['author_flair_css_class'] == 'male'):
            AMgender.append(1)
        else:
            AMgender.append(0)
    else:
        subreddit.append('AW')
        AWbody.append(temp['body'])
        if (temp['author_flair_css_class'] == 'male'):
            AWgender.append(1)
        else:
            AWgender.append(0)
            
print("Number of AskMen comments:", len(AMbody), len(AMgender))
print("Number of AskWomen comments:", len(AWbody), len(AWgender))
print("Total comments:" , len(AWbody) + len(AMbody))

Number of AskMen comments: 809290 809290
Number of AskWomen comments: 763614 763614
Total comments: 1572904


In [5]:
print("The total amount of men is", sum(gender))
print("The total amount of women is", 1572904-sum(gender))

print("The total amount of men in Ask Men is", sum(AMgender))
print("The total amount of women in Ask Men is", 809290 -sum(AMgender))

print("The total amount of men in Ask Women is", sum(AWgender))
print("The total amount of women in Ask Women is", 763614 -sum(AWgender))

The total amount of men is 863541
The total amount of women is 709363
The total amount of men in Ask Men is 699471
The total amount of women in Ask Men is 109819
The total amount of men in Ask Women is 164070
The total amount of women in Ask Women is 599544


---------

In [6]:
# Extract meaningful data (total)
vectorizer = CountVectorizer()
apple = vectorizer.fit_transform(body)

In [7]:
# Split data
# Divide into train and test
X_trainS, X_test, y_trainS, y_test = train_test_split(apple, gender, test_size=0.2, random_state=42)
# Divide into train and validation
X_train, X_valid, y_train, y_valid = train_test_split(X_trainS, y_trainS, test_size=0.2, random_state=42)

In [ ]:
#DOWNSIZE and then split data
#Discard "traash"
X_keep, trash1, y_keep, trash2 = train_test_split(apple, gender, test_size=0.995, random_state=42)
print(X_keep.shape, len(y_keep))
# Divide into train and test
X_trainS, X_test, y_trainS, y_test = train_test_split(X_keep, y_keep, test_size=0.2, random_state=42)
# Divide into train and validation
X_train, X_valid, y_train, y_valid = train_test_split(X_trainS, y_trainS, test_size=0.2, random_state=42)
print(X_train.shape, len(y_train))

---------

In [ ]:
# Extract meaningful data (gendered)
vectorizer = CountVectorizer()
apple = vectorizer.fit(AMbody + AWbody)
bodyAM = vectorizer.transform(AMbody)
bodyAW = vectorizer.transform(AWbody)

In [ ]:
# Split data (Gendered Male Training)
X_train, X_valid, y_train, y_valid = train_test_split(bodyAM, AMgender, test_size=0.2, random_state=42)
X_test, y_test = bodyAW, AWgender

In [ ]:
# Split data (Gendered Female Training)
X_train, X_valid, y_train, y_valid = train_test_split(bodyAW, AWgender, test_size=0.2, random_state=42)
X_test, y_test = bodyAM, AMgender

In [ ]:
print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))
print(apple.shape)

---------

In [8]:
# Classification time!
from sklearn.linear_model import LogisticRegression
lgClassifier = LogisticRegression(solver='lbfgs')
lgClassifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [9]:
# Classification time!
from sklearn.naive_bayes import MultinomialNB
mnbClassifier = MultinomialNB() 
mnbClassifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
# Validation time!
from sklearn.model_selection import cross_val_score

scoresLG2 = cross_val_score(lgClassifier, X_valid, y_valid, cv=10)
print("Logistic regression:", scoresLG2)
scoresNMB2 = cross_val_score(mnbClassifier, X_valid, y_valid, cv=10)
print("M. naive Bayes:", scoresNMB2)

Logistic regression: [0.65948824 0.66087889 0.65379266 0.65489152 0.65703727 0.66220297
 0.65914329 0.65600413 0.65501073 0.65667965]
M. naive Bayes: [0.6481246  0.64768754 0.64723646 0.65020265 0.64869268 0.64809664
 0.64527537 0.64905031 0.64622904 0.64436144]


In [11]:
# Test time!
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


y_pred = lgClassifier.predict(X_test)
print("Logistic Regression--")
print("Accuracy:",accuracy_score(y_test,y_pred))
print("Precision:",precision_score(y_test,y_pred))
print("Recall:",recall_score(y_test,y_pred))
print("F1-score:",f1_score(y_test,y_pred))
auc = roc_auc_score(y_test,y_pred)
print('Area under ROC (ROC AUC) is: {}'.format(auc))

print()
print()

y_pred = mnbClassifier.predict(X_test)
print("Multinomial Naive Bayes--")
print("Accuracy:",accuracy_score(y_test,y_pred))
print("Precision:",precision_score(y_test,y_pred))
print("Recall:",recall_score(y_test,y_pred))
print("F1-score:",f1_score(y_test,y_pred))
auc = roc_auc_score(y_test,y_pred)
print('Area under ROC (ROC AUC) is: {}'.format(auc))

Logistic Regression--
Accuracy: 0.668403368289884
Precision: 0.6605154954065929
Recall: 0.8142280355208509
F1-score: 0.7293610352950943
Area under ROC (ROC AUC) is: 0.6526441339985374


Multinomial Naive Bayes--
Accuracy: 0.653831604578789
Precision: 0.6728427538550825
Recall: 0.7185847269609745
F1-score: 0.6949618765371235
Area under ROC (ROC AUC) is: 0.6468337512930973


---------